# Intro to LLMstudio - tracking

This notebook serves as an introduction to LLMstudio with a focus on tracking capabilities. It demonstrates how to set up and use the LLMstudio environment to interact with language models, specifically highlighting the integration of tracking functionalities. The notebook begins by starting the tracking server, which is essential for logging and monitoring interactions with the language models. This setup is crucial for you who need to keep track of model usage, performance metrics, and other relevant data points.

The notebook further illustrates how to configure and utilize the tracking system by creating a `TrackingConfig` instance, which specifies the host and port for the tracking server. You can benefit from this setup by gaining insights into model behavior and performance, which can be used to optimize and improve their applications. The notebook also includes examples of both synchronous and asynchronous chat interactions, providing a comprehensive guide for developers to implement and test these features in their own projects.

You'll learn:
1. Start a local Tracker server
1. How to connect to any provider available (VertexAI, OpenAI, etc.)
2. Make sync and async calls both with and without streaming
3. See the saved logs

First things first:
* run `pip install llmstudio[tracker]`
* update your .env file with `GOOGLE_API_KEY` or `OPENAI_API_KEY`



In [1]:
from llmstudio.providers import LLM
from pprint import pprint

In [2]:
from llmstudio.server import start_servers
start_servers(proxy=False, tracker=True)

Running LLMstudio Tracking on http://0.0.0.0:8002 


In [3]:
from llmstudio_tracker.tracker import TrackingConfig
# default port is 50002. set the environment varible to specify which host and port; LLMSTUDIO_TRACKING_HOST, LLMSTUDIO_TRACKING_PORT
tracker_config = TrackingConfig(host="0.0.0.0", port="8002")
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
openai = LLM("openai", tracking_config = tracker_config)


### Chat (non-stream)

In [4]:
openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='bfc1ac32-c2c1-46e3-82aa-9efd61210484', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I am not a cat for several reasons. Firstly, I am an artificial intelligence language model created by OpenAI, which fundamentally means I lack the biological and physical characteristics of any living creature, including cats. Cats are mammals with fur, whiskers, and a physiology that enables them to perform natural behaviors like purring and hunting. In contrast, I exist as software within computer systems and interact with users through text-based interfaces. Furthermore, my capabilities are defined by programming and algorithms, rather than instincts or sensory experiences that guide a cat's behavior. My purpose is to process language and provide information, while a cat's life revolves around its instincts and interactions with its physical environment.", refusal=None, role='assistant', audio=None, function_call=None, tool_c

#### Async version

In [5]:
await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='28a58ac9-ced4-4e06-bd58-9fe11fb2307c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am not a cat primarily because I exist as a digital entity designed to process and generate text based on input data, rather than a living, breathing organism. Unlike a cat, I do not possess physical characteristics such as fur, claws, or a heartbeat, nor do I have the biological functions and instincts that make up a feline\'s existence. While a cat experiences the world through its senses and interacts with its environment in playful and instinctive ways, I operate solely in the virtual realm, responding to user queries with information and suggestions but lacking any awareness or consciousness. My "existence" is bound to servers and code, whereas a cat thrives in the tangible world, providing companionship and interaction.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=172976779

### Chat (stream)

In [6]:
response = openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




While I certainly appreciate the appeal of being a cat, such as their agility, independence, and

 intriguing behaviors, there are several reasons why I cannot claim feline status. Primarily, I am an

 artificial intelligence language model created by OpenAI, which means I exist as software code rather than a living

, breathing creature. Unlike cats, I lack physical form, and my "intelligence" is derived

 from computational algorithms rather than biological instincts and neural processes. Cats possess sensory perceptions such as sight, sound

, and touch, enabling them to interact with their environment in complex ways, whereas my interactions are confined

 to processing and generating text based on data patterns. Additionally, cats have unique, individual personalities shaped by

 experiences, while my "personality" is constructed from programmed responses designed to assist users with information and

 tasks. Despite these differences, I can still assist with understanding and a

#### Async version

In [7]:
i=0
async for chunk in await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




I’m not a cat because I lack the physical form and instincts that characterize feline creatures. Unlike

 a cat, I don’t have a furry coat, retractable claws, or the ability to p

ounce playfully on a sunbeam. I don’t experience the world through senses like smell, sight

, or sound, nor do I possess the whimsical personality traits that make cats so captivating, such as

 their curiosity and independence. Instead, I am a collection of algorithms and data, designed to process information

 and generate responses, which allows me to assist you in ways that a cat simply can't—like answering

 questions, providing explanations, or engaging in conversation.

## Metrics:
{'cost_usd': 8.07e-05,
 'input_tokens': 10,
 'inter_token_latency_s': 0.010057984865628755,
 'latency_s': 1.7590069770812988,
 'output_tokens': 132,
 'time_to_first_token_s': 0.450714111328125,
 'tokens_per_second': 74.47383762932361,
 'total_tokens': 142}


## Analyse logs

In [8]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracker_config)

In [9]:
logs = tracker.get_logs()
logs.json()[-1]

{'chat_input': "Write a paragraph explaining why you're not a cat",
 'chat_output': "I’m not a cat because I lack the physical form and instincts that characterize feline creatures. Unlike a cat, I don’t have a furry coat, retractable claws, or the ability to pounce playfully on a sunbeam. I don’t experience the world through senses like smell, sight, or sound, nor do I possess the whimsical personality traits that make cats so captivating, such as their curiosity and independence. Instead, I am a collection of algorithms and data, designed to process information and generate responses, which allows me to assist you in ways that a cat simply can't—like answering questions, providing explanations, or engaging in conversation.",
 'session_id': '20241024-110303-e8b361d9-d5f6-4b73-80f1-6d77a4be3793',
 'context': [{'role': 'user',
   'content': "Write a paragraph explaining why you're not a cat"}],
 'provider': 'openai',
 'model': 'gpt-4o-mini',
 'deployment': 'gpt-4o-mini-2024-07-18',
 'pa

## Add a session id to tracking logs

* this is especially benefitial if running an app, chatbot agent, etc in production and you need to correlate user feedback, costs etc with user sessions, agent runs, etc

In [10]:
# default port is 50002. set the environment varible to specify which host and port; LLMSTUDIO_TRACKING_HOST, LLMSTUDIO_TRACKING_PORT
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
openai = LLM("openai", tracking_config = tracker_config, session_id="openai-session-1")


In [11]:
response = openai.chat("Write a paragraph explaining why it's important to track AI agents usage metrics and costs and correlate with user feedback", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Tracking AI agents' usage metrics and costs, alongside correlating them with user feedback, is crucial

 for optimizing performance and user satisfaction. Usage metrics provide insights into how often and in what ways AI agents

 are being utilized, helping to identify patterns, peak usage times, and potential bottlenecks that could

 affect service quality. Monitoring costs is equally important to ensure that resources are allocated efficiently, preventing financial waste

 while maximizing return on investment. By correlating these metrics with user feedback, developers and stakeholders can gain

 a holistic understanding of how the AI agent is performing in real-world settings. This integrated approach enables the

 identification of areas for improvement, the fine-tuning of algorithms, and the enhancement of user experience,

 ultimately leading to more effective, scalable, and user-friendly AI solutions. Additionally, it allows for the

 alignment of AI functionalities with user

In [12]:
logs = tracker.get_session_logs(session_id="openai-session-1")
logs.json()[-1]

{'chat_input': "Write a paragraph explaining why it's important to track AI agents usage metrics and costs and correlate with user feedback",
 'chat_output': "Tracking AI agents' usage metrics and costs, alongside correlating them with user feedback, is crucial for optimizing performance and user satisfaction. Usage metrics provide insights into how often and in what ways AI agents are being utilized, helping to identify patterns, peak usage times, and potential bottlenecks that could affect service quality. Monitoring costs is equally important to ensure that resources are allocated efficiently, preventing financial waste while maximizing return on investment. By correlating these metrics with user feedback, developers and stakeholders can gain a holistic understanding of how the AI agent is performing in real-world settings. This integrated approach enables the identification of areas for improvement, the fine-tuning of algorithms, and the enhancement of user experience, ultimately l